## Running samtools view via the WES API

#### Learning Objectives
Workshop attendees will learn how use the GA4GH Workflow Execution Service (WES).  

What will participants do as part of the exercise?

 - Understanding how to run a workflow via WES
 - Adjust some parameters of the workflow
 - Check the status of the runs
 - Access the workflow results via DRS
 
 #### Icons in this Guide

 🖐 A hands-on section where you will code something or interact with the server
 
 
 
Just as we used a python client to submit DRS requests in the previous notebook we will use a similar client from the fasp package to run workflows.

By setting the debug flag to True on the client, the actual http calls will be shown.

#### Step 1: Set some variables for your specific settings
 🖐 Set SB_PROJECT to the username and project you created earlier.
You will only need to change SB_API_KEY_PATH if you changed the location of the key file.


In [1]:
from fasp.workflow import sbcgcWESClient
SB_PROJECT = 'username/project-name'
SB_API_KEY_PATH = '~/.keys/sbcgc_key.json'
DOWNLOAD_LOCATION = '~/Downloads'

cl = sbcgcWESClient(SB_PROJECT, SB_API_KEY_PATH, debug=False)

#### Step 2: Set up workflow run
Note that we may have to use one of the alternate options below for the SAMTools View application.

In [ ]:
task_name = "Tutorial run 1 test via WES - header only"

drs_uris  = ['drs://cgc-ga4gh-api.sbgenomics.com/62b0a8daf08fea4770469454',
'drs://cgc-ga4gh-api.sbgenomics.com/62bd910d14b0e420a0e2dc8f']

#sam_view_app = 'sbg://admin/sbg-public-data/samtools-view-1-9-cwl1-0'
#sam_view_app = 'sbg://yasasvinip/test-1/samtools-view-1-9-cwl1-0'
sam_view_app = 'sbg://forei/ismb-tutorial/samtools-view-1-9-cwl1-0-tut'
#sam_view_app = 'sbg://ianfore/ian-tutorial/samtools-view-1-9-cwl1-0-tut'

params = {
"project": SB_PROJECT,
"name": task_name,
"inputs": {
    "output_header_only": True,
    "include_header": True,
        "in_alignments": {
          "path": drs_uris[0],
          "class": "File"
        }
    }
}

#### Step 3: submit the workflow 

Now we have formulated the body in the way that it can be passed to a client function as follows.

In [ ]:
import json



run_id= cl.run_generic_workflow(
    workflow_url=sam_view_app,
    workflow_params = json.dumps(params),
    workflow_type = "CWL",
    workflow_type_version = "v1.0",
    verbose=False
)
run_id

Step 4: Check the status of the workflow

In [ ]:
cl.get_task_status(run_id)

The task will take a few minutes to complete.

Go ahead with the next step

#### Step 5: Adjust a parameter of the tool

Using the desciption of the app on the Seven Bridges Platform
identify the parameter that directs samtools view to only output the count of matching records
https://cgc.sbgenomics.com/u/forei/ismb-tutorial/apps/#forei/ismb-tutorial/samtools-view-1-9-cwl1-0

Alter the details in the following copy of the previous run
* Edit the parameters section below to set the value of the parameter you have identified to True.
* Delete the other parameters from the previous run.
* Enter a task name that will help you identify the task

In [ ]:
task_name2 = "new copy fixed samtools view count only"

params2 = {
"project": SB_PROJECT,
"name": task_name2,
"inputs": {
    "count_alignments": True,
        "in_alignments": {
          "path": drs_uris[0],
          "class": "File"
        }
    }
}

#### Step 6: Submit the revised task and make a note of the run_id

In [ ]:
run_id2 = cl.run_generic_workflow(
    workflow_url=sam_view_app,
    workflow_params = json.dumps(params2),
    workflow_type = "CWL",
    workflow_type_version = "v1.0",
    verbose=False
)
run_id2

#### Step 7: Check the status of the second run
🖐 Noting the name of the variable in which the id of the new run was executed, write a line to check the status of the run

In [ ]:
# add code below


## Getting the results - via DRS

Once the first run is complete, further steps can use DRS to obtain the file output from the workflow.

#### Step 8: Show the full response from the WES server
Note the outputs section which shows outputs specitic to the SAMTools View command.

In [ ]:
runLog = cl.get_run_log(run_id)
runLog

#### Step 9: Extract the value of the DRS URI for the output file

In [ ]:
results_drs_uri = runLog['outputs']['out_alignments']['path']
results_drs_uri

We'll pass over the question of how one would determine which DRS server that URI needs to be sent to because
* In this case it's fairly obvious - it's the CGC DRS Server
* We want to get something up and working

#### Step 10 Use DRS to get file details: 
First we set up a DRS client to access the results files

In [ ]:
from fasp.loc import sbcgcDRSClient
drsClient = sbcgcDRSClient(SB_API_KEY_PATH, 's3')

As in Example 2-1 DRS can be used to get details of the file. Note that only the id portion of the DRS URI is being passed. 

In [ ]:
# get the id part of the URI
out_alignments_drs_id = results_drs_uri.split('/')[-1]
print(f"Getting {out_alignments_drs_id} from DRS Client")
fileDetails = drsClient.get_object(out_alignments_drs_id)
fileDetails

#### Note we can check the size of results files before downloading

In [ ]:
print(fileDetails['size'])

#### Step 11: Downloading the file
Now we can use the url obtained to download the file. 

DRS is used again to get a URL to the results file.

In [ ]:
url = drsClient.get_access_url(out_alignments_drs_id,'s3')

We'll create a small function to encapsulate the download.

In [ ]:
import requests
import os
def download(url, file_path):
    with open(os.path.expanduser(file_path), "wb") as file:
        response = requests.get(url)
        file.write(response.content)

In [ ]:
fullPath = DOWNLOAD_LOCATION + '/' + fileDetails['name']
download(url, fullPath)

#### Step 12: Repeat the steps above to retrieve the results of the second run
...perhaps by defining a function to perform the same steps

In [ ]:
# code here